In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import sqlite3
import requests 
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
# Build connection
connection=sqlite3.connect('../input/california-traffic-collision-data-from-switrs/switrs.sqlite')

In [ ]:
# Display columns of collisions table
query = "SELECT collision_time, collision_date, county_location, county_city_location, collision_severity, killed_victims, injured_victims FROM collisions"
collisions = pd.read_sql_query(query,connection)

In [ ]:
# Display table
collisions.head()

In [ ]:
# Display info
collisions.info()

In [ ]:
# Display nulls
collisions.isna().sum()

In [ ]:
# Drop nulls in collision_time column and fill other nulls with 0
collisions = collisions.dropna(subset=["collision_time"])
collisions = collisions.fillna(0)

In [ ]:
# Display nulls
collisions.isna().sum()

In [ ]:
# Display unique values under the collision_severity column
collisions["collision_severity"].unique()

In [ ]:
# Display unique values under the county_location column
np.sort(collisions["county_location"].unique())

In [ ]:
# Display unique values under the county_city_location column
collisions["county_city_location"].unique()

In [ ]:
# Convert to pandas datetime
collisions["collision_date"] = pd.to_datetime(collisions["collision_date"])

In [ ]:
# Get FIPS codes of counties
url = "https://en.wikipedia.org/wiki/List_of_counties_in_California#List"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(url)

# Parse table
soup = BeautifulSoup(response.text, 'html.parser')
california_counties=soup.find('table',{'class':"wikitable"})

# Load table to pandas df
california_counties=pd.read_html(str(california_counties))
california_counties=pd.DataFrame(california_counties[0])

# Select columns
california_counties = california_counties[["County", "FIPS code[5]"]]

# Rename columns
california_counties = california_counties.rename(
    columns={
        'County': 'county', 
        'FIPS code[5]': 'FIPS'
    }
)

# Convert FIPS column to string
california_counties["FIPS"] = california_counties["FIPS"].astype('str')

# Convert column to plotly-compatible FIPS format
california_counties["FIPS"] = california_counties["FIPS"].apply(lambda x: x.zfill(3))
california_counties["FIPS"] = "06"+california_counties["FIPS"]

# Lowercase and remove county word in county column
california_counties["county"]=california_counties['county'].str.lower()
california_counties["county"] = california_counties["county"].str.replace(
    " county",
    ""
)

In [ ]:
# Show data
california_counties

In [ ]:
# Show data info
collisions.info()

In [ ]:
# Rename county_location
collisions = collisions.rename(
    columns={
        'county_location': 'county'
    }
)

In [ ]:
# Combine collisions and california_counties
collisions = pd.merge(collisions, california_counties, on='county')

In [ ]:
# Show data info
collisions.info()

In [ ]:
# Create hour, month and year columns
collisions["collision_hour"] = pd.to_datetime(collisions["collision_time"]).dt.strftime('%H').astype(int)
collisions["collision_month"] = collisions["collision_date"].dt.month
collisions["collision_year"] = collisions["collision_date"].dt.year
collisions["collision_day"] = collisions["collision_date"].dt.dayofweek

In [ ]:
# Save file
collisions.to_csv("collisions.csv")